In [1]:
# processing the dictionary objects

In [2]:
# altair to visualize the data. read in pickle
# determine df shape for this to work
# import birdseye and snoop
import snoop
import altair as alt
# from vega_datasets import data
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr, BaseModel
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx, extract_features #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
from pydantic_fula_dictionary_entries import dict_entry, trustyGet, getAssert, parse_dataframe_groups #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial
import pickle
import openpyxl

In [3]:
# b = [True, True, False]
# s = ['a','b','c']
# text = ''.join(chain(compress(s,b))).strip()
# print(text)
# print(list(compress(s,b)))

## Loading Data

In [4]:
with open('pickled_results/all_dictionary_entries_list.pkl', 'rb') as file:
   all_entries = pickle.load(file, encoding='utf-8')

### Checking data contents

In [57]:
ct = 0
# entry
for e in all_entries:
   if len(e.irregularities) > 0:
      print(e.irregularities)
      ct+=1
      print(e.lemmaLine_runs)
print(ct)

['What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous']
['bananaare ', '(nde), ', 'banaanaaru ', '(ndu) / bananaaje (ɗe), bananaaji (ɗi)  ', 'ZH  Z<->  [bordiya]']
['What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous']
['bolonngal ', '(ngal), ', 'bolonnga ', '(nga) / bolooɗe (ɗe)  ', 'DCZeiLqY  CZ<+>  [bolaangal]:ZI  [bolongal]:F  [bolonru, bolooru]:ZE  [kirhirdi]']
['What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous']
['colla ', '(o), ', 'collal ', '(ngal)  ', 'DF  cf.:[mbulwuldi]']
['What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possi

In [6]:
dialects = []
typ_df = pd.DataFrame()
feat_df = pd.DataFrame()
for i,e in enumerate(all_entries):
   typ_dct = {}
   for k,v in e.dict().items():
      typ_dct[k]=type(v)
   tdf = pd.Series(typ_dct).to_frame().T
   fdf = pd.Series(e.dict()).to_frame().T
   typ_df = pd.concat([typ_df,tdf],ignore_index=True)
   feat_df = pd.concat([feat_df,fdf],ignore_index=True)


In [7]:
rare_cols = typ_df.describe().T
rare_cols = rare_cols[rare_cols['unique']>1]
rare_cols = rare_cols[rare_cols['top'].isin([type(True)])].T
# rare_cols

unused_cols = typ_df.describe().T
unused_cols = unused_cols[unused_cols['unique']==1]
unused_cols = unused_cols[unused_cols['top'].isin([type(True)])]
unused_cols = unused_cols.T
unused_cols

,root_origin,FulaSenseEnglish,FulaSenseEnglish_Count,FulaSenseEnglish_Annotations,FulaSenseEnglish_Synonyms,FulaSenseEnglish_unusedContent,FulaSenseFrench,FulaSenseFrench_Count,FulaSenseFrench_Annotations,FulaSenseFrench_Synonyms,FulaSenseFrench_unusedContent,FulaSynonyms,FulaCrossRef
count,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985
unique,1,1,1,1,1,1,1,1,1,1,1,1,1
top,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>,<class 'bool'>
freq,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985,7985


In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
df = feat_df[feat_df.columns.difference(unused_cols.columns)]
# df = df.set_index('lemma_index')
df.explode('FulaDialects')

## Exploring Dialects Field

In [9]:

multi_len = df['FulaDialects'].apply(lambda x: x if isinstance(x,list) and len(x)>1 else None)
multi_len = multi_len[multi_len.notna()]
# print(df.shape)
# df.explode('FulaDialects')['FulaDialects']
multi_len

8                  [-, +]
9               [G,A, FT]
13           [-, FJ,FT,M]
28             [FJ,M, FJ]
31               [-, A,G]
              ...        
7924       [FJ,M,G, FJ,G]
7951             [G,A, G]
7959    [-, FT,M,FJ,A,DO]
7963         [G,A, M,G,A]
7978           [FJ,M, FJ]
Name: FulaDialects, Length: 576, dtype: object

In [10]:
temp = df['FulaDialects'].apply(lambda x: ','.join(x))
temp = temp.apply(lambda x: x.split(','))
temp

0          [-]
1       [Mali]
2          [-]
3       [Mali]
4       [Mali]
         ...  
7980    [Mali]
7981    [Mali]
7982    [Mali]
7983    [Mali]
7984    [Mali]
Name: FulaDialects, Length: 7985, dtype: object

In [11]:
lists = temp.apply(lambda x: isinstance(x,list))
lens = temp.apply(lambda x: len(x))
# multi_len = multi_len[multi_len.notna()]
# print(df.shape)
print(lists.describe())
print(lens.describe())
# df.explode('FulaDialects')['FulaDialects']
# multi_len.sample(1).values

count     7985
unique       1
top       True
freq      7985
Name: FulaDialects, dtype: object
count    7985.000000
mean        1.488040
std         1.316569
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        13.000000
Name: FulaDialects, dtype: float64


## Processing Dialect field contents

In [20]:
dialected_df = df.copy()
dialected_df['FulaDialects'] = dialected_df['FulaDialects'].apply(lambda x: ','.join(x)).apply(lambda x: x.split(','))


,FulaDialects,lemma_index
0,-,5
1,Mali,8
2,-,11
3,Mali,14
4,Mali,17
...,...,...
7980,Mali,32488
7981,Mali,32492
7982,Mali,32496
7983,Mali,32500


### Calculating Counts

In [ ]:
dialected_df = dialected_df.explode('FulaDialects')
counts = dialected_df[['FulaDialects','lemma_index']]
counts.drop_duplicates()

In [13]:
dialected_df[dialected_df['FulaDialects']=='FJ']

,FulaAnnotations_runs,FulaDialects,FulaNoun_NounsAndClass,FulaNoun_PluralsAndClass,FulaPOSClass,FulaPOSTags,englishGlossLine_runs,frenchGlossLine_runs,irregularities,lemma,lemmaLine_runs,lemmaLine_unusedContent,lemma_index,paragraphs,root,root_metadata,root_subpiece
13,True,FJ,"[(aadamanke(ejo), o)]","[(o) , aadamankeeɓe (ɓe) ]",[Noun],True,[human being],[être humain],[],aadamanke(ejo),"[aadamanke(ejo) , (o) / aadamankeeɓe (ɓe) , D...","[(1, (o) / aadamankeeɓe (ɓe) ), (2, DCZ C<FJ...",47,"[{'para_first_line_indent': -91440, 'paragraph...",AADAMA,"{'root_index': 43, 'root_runs': ['AADAMA']}",False
28,True,FJ,False,True,[Verb],[v.av ],"[to lose one's head; to scream loudly, cry out]","[s'affoler(DH); crier très fort(D), pousser un...",[],aatude,"[aatude , v.av , DcZH C<FJ>,Z<FJ,M> cf.:[h...","[(1, v.av ), (2, DcZH C<FJ>,Z<FJ,M> cf.:[ha...",110,"[{'para_first_line_indent': -91440, 'paragraph...",AAT-,"{'root_index': 109, 'root_runs': ['AAT-']}",False
28,True,FJ,False,True,[Verb],[v.av ],"[to lose one's head; to scream loudly, cry out]","[s'affoler(DH); crier très fort(D), pousser un...",[],aatude,"[aatude , v.av , DcZH C<FJ>,Z<FJ,M> cf.:[h...","[(1, v.av ), (2, DcZH C<FJ>,Z<FJ,M> cf.:[ha...",110,"[{'para_first_line_indent': -91440, 'paragraph...",AAT-,"{'root_index': 109, 'root_runs': ['AAT-']}",False
37,True,FJ,"[(abajada, abajadda, o)]","[(o) , abajadaaji (ɗi) ]",[Noun],True,[alphabet],[alphabet],[],"abajada, abajadda","[abajada, abajadda , (o) / abajadaaji (ɗi) , ...","[(1, (o) / abajadaaji (ɗi) ), (2, DcZ C<FJ>,...",146,"[{'para_first_line_indent': -91440, 'paragraph...",ABAJADDA,"{'root_index': 145, 'root_runs': ['ABAJADDA ...",False
42,True,FJ,False,True,[Verb],[v.pv ],"[to be first, precede; to promise]","[être le premier, précéder, devancer; promettr...",[],adaade,"[adaade , v.pv , DFZ C<FJ>,Z<-> [artaade]]","[(1, v.pv ), (2, DFZ C<FJ>,Z<-> [artaade])]",170,"[{'para_first_line_indent': -91440, 'paragraph...",AD- (1),"{'root_index': 169, 'root_runs': ['AD- (1)']}",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934,True,FJ,False,True,[Verb],[v.av ],"[to take, carry a child; to take; to set uprig...","[prendre, porter un enfant(D); prendre(Z); rel...",[],ƴettude,"[ƴettude , v.av , DCZ CZ<FJ,FT,M,BF> [ƴept...","[(1, v.av ), (2, DCZ CZ<FJ,FT,M,BF> [ƴeptud...",32306,"[{'para_first_line_indent': -91440, 'paragraph...",ƳETT-,"{'root_index': 32305, 'root_runs': ['ƳETT-']}",False
7959,True,FJ,"[(ƴoyre, ƴoƴre, nde)]","[ (nde) , ƴoƴe (ɗe) ]",[Noun],True,"[mischief, cunning, cleverness; scheme(C); way...","[malice, ruse, adresse; intrigue(CZ); manière ...",[],"ƴoyre, ƴoƴre","[ƴoyre, ƴoƴre , (nde) / ƴoƴe (ɗe) , DFCZ ...","[(1, (nde) / ƴoƴe (ɗe) ), (2, DFCZ C<->,Z<...",32408,"[{'para_first_line_indent': -91440, 'paragraph...",ƳOƳ -,"{'root_index': 32401, 'root_runs': ['ƳOƳ -']}",False
7968,True,FJ,"[(ƴumndi, ƴunndi, ndi)]",True,[Noun],True,[soot],[suie],[],"ƴumndi, ƴunndi","[ƴumndi, ƴunndi , (ndi) , CZ <FJ,FT,M,BF> [...","[(1, (ndi) ), (2, CZ <FJ,FT,M,BF> [nduuñu])]",32444,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUMNDI,"{'root_index': 32443, 'root_runs': ['ƳUMNDI']}",False
7978,True,FJ,"[(ƴuugaaɗo, o)]","[(o) , ƴuugaaɓe (ɓe) ]",[Noun],True,[hunchback],[bossu],[],ƴuugaaɗo,"[ƴuugaaɗo , (o) / ƴuugaaɓe (ɓe) , cZ C<FJ>,Z...","[(1, (o) / ƴuugaaɓe (ɓe) ), (2, cZ C<FJ>,Z<F...",32482,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUG-,"{'root_index': 32481, 'root_runs': ['ƳUUG-']}",False


In [23]:
dialected_df

,FulaAnnotations_runs,FulaDialects,FulaNoun_NounsAndClass,FulaNoun_PluralsAndClass,FulaPOSClass,FulaPOSTags,englishGlossLine_runs,frenchGlossLine_runs,irregularities,lemma,lemmaLine_runs,lemmaLine_unusedContent,lemma_index,paragraphs,root,root_metadata,root_subpiece
0,True,-,False,True,True,"[prn, sbj, sf ]",[you (sg.)],[tu],[],a,"[a , prn,sbj,sf , DFZH Z<->]","[(1, prn,sbj,sf ), (2, DFZH Z<->)]",5,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
1,True,Mali,False,True,True,"[suf, pos ]",[your (sg.) (only with certain nouns such as t...,"[ton, ta, tes (seulement pour certains substan...",[],-a,"[-a , suf,pos , F]","[(1, suf,pos ), (2, F)]",8,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
2,True,-,False,True,True,"[prn, ind ]",[you (sg.) (emphatic)],[toi (emphatique)],[],aan,"[aan , prn,ind , DFZH [an]:Z<->]","[(1, prn,ind ), (2, DFZH [an]:Z<->)]",11,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
3,True,Mali,False,True,True,"[prn, sbj, lf ]",[you (sg.)],[tu],[],aɗa,"[aɗa , prn,sbj,lf , DFZH [aɗo]:Z]","[(1, prn,sbj,lf ), (2, DFZH [aɗo]:Z)]",14,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
4,True,Mali,False,True,True,"[prn, sbj ]",[you (sg.) (used with imperfect and subjunctiv...,[tu (employé avec les formes imparfaites et su...,[],"-ɗaa, -ɗa","[-ɗaa, -ɗa , prn,sbj , DF]","[(1, prn,sbj ), (2, DF)]",17,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7980,True,Mali,"[(ƴuugo, ngo)]","[(ngo) , ƴuuge (ɗe) ]",[Noun],True,[hump],[bosse],[],ƴuugo,"[ƴuugo , (ngo) / ƴuuge (ɗe) , Z [naɗɗire]]","[(1, (ngo) / ƴuuge (ɗe) ), (2, Z [naɗɗire])]",32488,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUG-,"{'root_index': 32481, 'root_runs': ['ƳUUG-']}",False
7981,True,Mali,"[(ƴuulde, nde)]",True,[Noun],True,"[boil, furuncle]",[furoncle],[],ƴuulde,"[ƴuulde , (nde) , D]","[(1, (nde) ), (2, D)]",32492,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUULDE,"{'root_index': 32491, 'root_runs': ['ƳUULDE']}",False
7982,True,Mali,False,True,[Verb],[v.mv ],[to incline; to rear up],[s'incliner; se cabrer],[],ƴuuwaade,"[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]","[(1, v.mv ), (2, D [ƴuubaade, ƴooraade])]",32496,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUW-,"{'root_index': 32495, 'root_runs': ['ƳUUW-']}",False
7983,True,Mali,False,True,[Verb],[v.av ],[to increase in volume],[augmenter de volume],[],ƴuuƴude,"[ƴuuƴude , v.av , D]","[(1, v.av ), (2, D)]",32500,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUƳ-,"{'root_index': 32499, 'root_runs': ['ƳUUƳ-']}",False


In [49]:
dialects_descriptions = pd.read_excel('test_data/abbr_pdf_text.xlsx')
dialects_descriptions = dialects_descriptions[['FulaDialects','pg_summarized_pdf_text','pdf_text']]

counts = dialected_df[['FulaDialects','lemma_index']]
counts = counts.drop_duplicates().groupby('FulaDialects').count()
# print(counts)
counts.columns = ['lemma_counts']
counts.to_csv('lemma_counts_by_dialect.csv',encoding='utf-8-sig')


counts = counts.merge(dialects_descriptions,how='inner',on='FulaDialects')


dialects_descriptions = counts.to_excel('test_data/abbr_pdf_text.xlsx',encoding='utf-8-sig')
counts

,FulaDialects,lemma_counts,pg_summarized_pdf_text,pdf_text
0,+,197,multiple forms have multiple cited dialects in...,"Indicates that the source (e.g. C ( +) , Z( +}..."
1,-,1557,no dialect in source citatiob,Indicates that the source whose abbreviation a...
2,?,1,NaN,NaN
3,A,366,Adamawa,Adamawa
4,A(Jam'aare),2,Cameroonian Adamawa,(zone foulaphone du Cameroun et sous-dialecte ...
5,B,15,Benin,"Benin, Benin"
6,BF,599,Burkina-Faso,Burkina-Faso
7,DO,159,eastern dialects,"dialectes orientaux, eastern dialects = (A,G,K..."
8,F,2,Ferlo - Senegal,Ferlo (une zone foulaphone du Senegal; a Fula-...
9,FJ,723,Fouta Djalon - Guinea,"Fouta Djalon, Futa Jalon"


### Exploring low-count dialects

In [56]:
# dialected_dct = {}
small_dialects = {}

# counts = dialected_df[['FulaDialects','lemma_index']]
# for k,v in counts.drop_duplicates().groupby('FulaDialects'):
for k,v in pd.Series(data = counts['lemma_counts'].values, index = counts['FulaDialects']).items():
   # print(type(v))
   # dialected_dct[k]=v
   # print(k,'\t',v)
   if v < 10:
      small_dialects[k]=v 
      print(k,'\t',v)


? 	 1
A(Jam'aare) 	 2
F 	 2
FN 	 8
Gimmballa 	 1
Gimmballa(Ñafunke) 	 2
K 	 3
Keñeba 	 4
Motti 	 1
S 	 5
Soosooɓe(Guumel) 	 1


In [15]:
sd = list(small_dialects.keys())
print(small_dialects)
# print(sd)
list(small_dialects[sd[9]].index)

{'?':     FulaDialects lemma_index
191            ?         815, "A(Jam'aare)":      FulaDialects lemma_index
49    A(Jam'aare)         195
5473  A(Jam'aare)       22792, 'F':      FulaDialects lemma_index
5296            F       21954
5581            F       23189, 'FN':      FulaDialects lemma_index
1122           FN        4848
1308           FN        5612
1940           FN        8092
4618           FN       19123
4743           FN       19649
6852           FN       28122
7388           FN       30216
7852           FN       31975, 'Gimmballa':     FulaDialects lemma_index
307    Gimmballa        1332, 'Gimmballa(Ñafunke)':             FulaDialects lemma_index
4226  Gimmballa(Ñafunke)       17488
4250  Gimmballa(Ñafunke)       17574, 'K':      FulaDialects lemma_index
1817            K        7597
1845            K        7707
2760            K       11382, 'Keñeba':      FulaDialects lemma_index
270        Keñeba        1159
696        Keñeba        3030
4651       Keñeba       

[1308, 2760, 4449, 6093, 7911]

In [16]:
dialects_to_drop = {
   '?':'not the only dialect mentioned: [baabaho]:C<G,N>,Z<?,N>',
}
# notes:
   #F:   (M.S. Diallo, 1990)<F>... Both instances are cited by this author.
      #specific
   #Gimmballa:    # 'beltol ?  n.  (Imperato, 1973)  <Gimmballa>  [pooɗol\xa0?]  cf.:[ɓelt-]'
      #specific
   #Gimmballa(Ñafunke): #  para_text='miɗo  prn,sbj,lf  DFZH  [miɗe]:ZH  [miɗa, meɗe]:H<Soosooɓe(Guumel)>  [ɓiɗo]:H<Gimmballa(Ñafunke)>'
         #para_text='miɗen  prn,sbj,lf  DFH  [ɓiɗe]:H<Gimmballa(Ñafunke)>'
      #just specified by Z and H
   #K    para_text='fotiire (nde) / potiije (ɗe)  DFCZ  CZ<M,K,NE>  [fayannde]'
      #just specified used by C and Z
   #  Keñeba   para_text='bareeru (ndu) / bareeji (ɗi)  DzhYo  Z<FJ>,H<Keñeba>,O<FJ>  [rawaandu]'
      # just specified by Z
   # Motti     para_text='seydi  H  <Motti>'
      # just specified by H
   # S   para_text='faa, fa  DFCZH  C<+>,Z<M,S>  [haa]:CZ<FJ,FT,NE,G>,H<FN>'
      #just specified used by C and Z
   #Soosooɓe(Guumel)    same as : Gimmballa(Ñafunke):       #  para_text='miɗo  prn,sbj,lf  DFZH  [miɗe]:ZH  [miɗa, meɗe]:H<Soosooɓe(Guumel)>  [ɓiɗo]:H<Gimmballa(Ñafunke)>'

   

## saving results to objects

In [17]:
all_entries[4449].paragraphs

[Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=18410, run_text=['mbabba ', '(mba) / bamɗi (ɗi)  ', 'DCZ  CZ<FJ,FT,M,NE,G,S>  [(pl.) baddi, bamli]:Z  [ndakiire, tefeewa, tefeere]'], para_text='mbabba (mba) / bamɗi (ɗi)  DCZ  CZ<FJ,FT,M,NE,G,S>  [(pl.) baddi, bamli]:Z  [ndakiire, tefeewa, tefeere]', run_font_size_pt=[None, None, 8.0], run_bold=[True, None, None], para_left_indent=46826035380000, run_italic=[None, True, None], run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt']),
 Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=18411, run_text=['donkey'], para_text='donkey', run_font_size_pt=[None], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], run_font_name=['TmsRmn 10pt']),
 Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=18412, run_text=['âne, ânesse'], para_text='âne, ânesse', run_font_size_pt=[None], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], 

In [72]:
len(all_entries)

7985

In [78]:
dialected_df = df.copy()
dialected_df['FulaDialects'] = dialected_df['FulaDialects'].apply(lambda x: ','.join(x)).apply(lambda x: x.split(','))

temp = pd.Series(data = dialected_df['FulaDialects'].values, index = dialected_df['lemma_index'])
# print(temp.index.drop_duplicates())
# temp[temp.index.()]
print(temp)

lemma_index
5           [-]
8        [Mali]
11          [-]
14       [Mali]
17       [Mali]
          ...  
32488    [Mali]
32492    [Mali]
32496    [Mali]
32500    [Mali]
32504    [Mali]
Length: 7985, dtype: object


In [79]:
with open('pickled_results/all_dictionary_entries_list.pkl', 'rb') as file:
   all_entries = pickle.load(file, encoding='utf-8')
print(type(all_entries[0]))
for i,dialects in pd.Series(data = dialected_df['FulaDialects'].values, index = dialected_df['lemma_index']).items():
   try:
      all_entries[i].FulaDialects = dialects
   except:
      print(dialects)
      print(i)
      print(all_entries[i])
   # print(all_entries[i])
   # print(e[1])
   # break

<class 'pydantic_fula_dictionary_entries.dict_entry'>
['Mali']
7989


IndexError: list index out of range

In [18]:
pd.reset_option("^display")